In [2]:
import numpy as np,os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, power_transform, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.pipeline import make_pipeline
import seaborn as sns,matplotlib.pyplot as plt

from library.utils import evaluate, read_data
from library.cleaners import kDN, ih_prob

In [3]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [ ]:
X,y_noisy,y_real = read_data(DATASETS[1],True)

In [ ]:
precision = precision_score(y_real,y_noisy)
recall = recall_score(y_real,y_noisy)
precision, recall

In [ ]:
n = kDN(X,y_noisy,K=5)
c = kDN(X,y_real,K=5)
n.mean(),c.mean(),'w',n.std(),c.std()

In [ ]:
plt.plot(range(len(n)),np.sort(n),'r');

In [ ]:
imbs = {
    'smote': SMOTE(k_neighbors=5),
    'rus': RandomUnderSampler('not minority'),
    'wilson':EditedNearestNeighbours(n_neighbors=5),  #Default was 3
}
clfs = {
    'nb': GaussianNB(),
    'rf': RandomForestClassifier(n_estimators=50),
}

models = {}
for im,samp in imbs.items():
    for c,clf in clfs.items():
        models[(im,c)] = make_pipeline(samp,clf)

In [14]:
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=42)
def pr_rec_score(y,yp):
    prec, rec, _ = precision_recall_curve(y,yp)
    return auc(rec,prec)
scorers = [matthews_corrcoef,pr_rec_score]
NOISE_TYPES = ['No','Bug','Non-Bug','Both']


In [ ]:
cols = pd.MultiIndex.from_product([NOISE_TYPES,imbs.keys(),clfs.keys(),[f.__name__ for f in scorers]],names=['noise','imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)

In [ ]:
for k in models:
    print(k)
    for d in DATASETS:
        X,y_noisy,y_real = read_data(d,stats=False)
        for noise_type in NOISE_TYPES:
            y = y_noisy.copy()
            if noise_type=='No':
                y = y_real.copy()
            elif noise_type=='Both':
                pass
            elif noise_type=='Bug':  #So recall 1.0
                y[y_real==1] = 1
            elif noise_type=='Non-Bug': #So precision 1.0
                y[y_real==0] = 0
            p = precision_score(y_real,y)
            r = recall_score(y_real,y)
                
            r = evaluate(models[k],X,y,y_real,cv,scorers)
            for f in r:
                df.loc[d,(noise_type,k[0],k[1],f)] = r[f].mean()
            #print(noise_type,df.loc[d,(noise_type,k[0],k[1],slice(None))])
            #print()

In [ ]:
df.to_csv("Impact of Types of Noise.csv")

## Analysis

## Notes
1. Both `pr_rec_score` & `matthews_corrcoef`: As expected, "No" noise performs best, while "Both" worst, in terms of whole avg.
2. Interestingly, "Bug" i.e. (So recall=1.0) always performs better than "Non-Bug" i.e. (So precision=1.0)
3. Avg Rank: pr_rec_score- `[1.74, 1.95, 2.82, 3.48]`, Mathew- `[1.78 , 2.46, 2.59, 3.17]`   (['No','Bug','Non-Bug','Both'] in that serial)
4. So for Mathew, in terms of Avg Rank, Non-bug is better, contradicting point 2

In [4]:
import numpy as np
import pandas as pd,os
from scipy.stats import wilcoxon, friedmanchisquare, rankdata
import scikit_posthocs as sp

In [11]:
df = pd.read_csv("Impact of Types of Noise.csv",header=[0,1,2,3],index_col=0)
df.isna().sum().sum()

0

In [20]:
tmp = df.drop(columns=['matthews_corrcoef'],level=3,axis=1).droplevel(3,axis=1)
tmp

noise                                     No                                \
imb                                    smote                 rus             
clf                                       nb        rf        nb        rf   
activemq-5.8.0.csv                  0.395876  0.322205  0.393498  0.299466   
groovy-1_6_BETA_1.csv               0.419579  0.623572  0.409148  0.507401   
activemq-5.3.0.csv                  0.466876  0.499462  0.463518  0.463055   
wicket-1.3.0-incubating-beta-1.csv  0.448159  0.452276  0.420954  0.439376   
jruby-1.1.csv                       0.628827  0.683200  0.622085  0.682370   
jruby-1.4.0.csv                     0.606393  0.632672  0.603336  0.591495   
lucene-2.3.0.csv                    0.735496  0.891819  0.696705  0.863578   
hbase-0.95.2.csv                    0.528984  0.576269  0.524307  0.580407   
lucene-3.0.0.csv                    0.335744  0.547999  0.333666  0.473437   
camel-2.9.0.csv                     0.359455  0.281933  0.354588  0.240935   
wicket-1.5.3.csv                    0.314838  0.213126  0.304197  0.194468   
lucene-3.1.csv                      0.262429  0.211465  0.237930  0.150720   
groovy-1_6_BETA_2.csv               0.420924  0.608471  0.390608  0.532414   
activemq-5.2.0.csv                  0.575813  0.618565  0.569416  0.574162   
groovy-1_5_7.csv                    0.476527  0.437702  0.380258  0.429272   
hbase-0.95.0.csv                    0.477637  0.672101  0.458310  0.640212   
camel-2.11.0.csv                    0.268834  0.320947  0.264564  0.214674   
jruby-1.7.0.preview1.csv            0.416942  0.324183  0.407108  0.326145   
hive-0.10.0.csv                     0.501290  0.578430  0.495199  0.515762   
camel-2.10.0.csv                    0.365491  0.305177  0.356703  0.251490   
derby-10.2.1.6.csv                  0.752979  0.815774  0.750818  0.801944   
jruby-1.5.0.csv                     0.538658  0.541807  0.529794  0.526005   
derby-10.3.1.4.csv                  0.650234  0.734835  0.650284  0.717577   
lucene-2.9.0.csv                    0.472418  0.580243  0.461033  0.538144   
hbase-0.94.0.csv                    0.578276  0.643803  0.556492  0.630061   
camel-1.4.0.csv                     0.517101  0.561527  0.522361  0.542517   
activemq-5.1.0.csv                  0.430904  0.409515  0.433551  0.373860   
activemq-5.0.0.csv                  0.605349  0.718797  0.596658  0.694515   
derby-10.5.1.1.csv                  0.507003  0.535254  0.512614  0.494910   
wicket-1.3.0-beta2.csv              0.417822  0.365879  0.401366  0.337944   
hive-0.12.0.csv                     0.238689  0.566989  0.234720  0.397570   
hive-0.9.0.csv                      0.628797  0.776453  0.625670  0.755785   

noise                                                        Bug            \
imb                                   wilson               smote             
clf                                       nb        rf        nb        rf   
activemq-5.8.0.csv                  0.422668  0.370918  0.397364  0.305907   
groovy-1_6_BETA_1.csv               0.416746  0.561341  0.422555  0.532286   
activemq-5.3.0.csv                  0.478188  0.478700  0.469061  0.494934   
wicket-1.3.0-incubating-beta-1.csv  0.465441  0.513826  0.448219  0.346060   
jruby-1.1.csv                       0.617425  0.696770  0.625469  0.629403   
jruby-1.4.0.csv                     0.598154  0.639718  0.603682  0.613493   
lucene-2.3.0.csv                    0.723203  0.842385  0.681853  0.874443   
hbase-0.95.2.csv                    0.546821  0.578973  0.529158  0.566680   
lucene-3.0.0.csv                    0.373376  0.482133  0.327060  0.421296   
camel-2.9.0.csv                     0.363022  0.275444  0.365244  0.254994   
wicket-1.5.3.csv                    0.330720  0.217922  0.320949  0.214964   
lucene-3.1.csv                      0.245012  0.252150  0.251160  0.148109   
groovy-1_6_BETA_2.csv               0.409422  0.563244  0.390198  0.569836   
activemq-5.2.0.csv   

In [21]:
dic = {}
for noise_type in NOISE_TYPES:
    dic[noise_type] = tmp[noise_type].values.reshape(-1)
    print(noise_type,dic[noise_type].mean())

No 0.4964820945632966
Bug 0.48525796620882905
Non-Bug 0.4615927690560883
Both 0.4367618136000903


In [22]:
friedmanchisquare(*(dic[a] for a in dic))

FriedmanchisquareResult(statistic=223.41250000000036, pvalue=3.6729655381200596e-48)

In [23]:
res = np.concatenate([dic[ntype].reshape(-1,1) for ntype in NOISE_TYPES],axis=1)
res.shape

(192, 4)

In [24]:
ranks = res.copy()
for i in range(res.shape[0]):
    ranks[i] = rankdata(1-res[i])

In [25]:
ranks.mean(axis=0)

array([1.74479167, 1.953125  , 2.81770833, 3.484375  ])